# Entregable Final del Reto

Ana Paula Katsuda, A01025303

Octubre 2020

# Introducción

Resulta de gran importancia comprender las erupciones volcánicas y los factores implicados en las mismas. Dicho conocimiento tiene un uso elemental al tomar decisiones durante una explosión volcánica e incluso tiene la posibilidad de salvar vidas. El uso de los videojuegos serios para rescatistas promueve la obtención de un entrenamiento informado y eficaz, por lo que el objetivo del presente escrito es hacer uso de la programación para contribuir al análisis de la trayectoria de proyectiles balísticos. Lo anterior, relacionado al volcán Popocatépetl, ubicado en Puebla y con el fin de proporcionar a los rescatistas un video juego serio que los prepare. 

# Metodología

A partir de lo discutido en clase, fue posible obtener relaciones matemáticas que logran modelar de manera precisa el movimiento de los balísticos del Popocatépetl. Las relaciones obtenidas fueron las siguientes:

**Trayectoria de Proyectil sin considerar resistencia al aire**

$\vec{r}(t) = v_{ox}(t)\vec{i} + (\frac{1}{2}gt^2 + v_{oy}t + s_{oy})\vec{j}$

Conociendo las aceleraciones (en este caso constantes) de los componentes $x$ y $y$, se integraron las mismas para obtener la velocidad, y posteriormente para obtener la posición de cada componente. En este caso, la aceleración del componente $x$ fue de 0 ya que la velocidad es constante, mientras que la aceleración del componente $y$ fue de $-9.81 \frac{m}{s^2}$ debido a la fuerza ejercida por la gravedad. 

Asimismo, fue necesario tomar en cuenta la resistencia al aire, por lo que se requirieron las siguientes relaciones:

**Resistencia al aire**

$\vec{F_D} = -Dv\vec{v}$

Lo anterior, sabiendo que la resistencia al aire ejerce una fuerza opuesta al movimiento.

**Constante D**

$D = \frac{CA\rho}{2}$

Donde $C$ es el coefficiente de arrastre, $A$ es el área frontal del proyectil y $\rho$ es la densidad del aire.

**Aceleración a partir de la segunda ley de Newton**

Esta se divide en los componentes puesto a que, para el componente $x$, solo se toma en cuenta la resistencia al aire mientras que para el componente $y$, se toma en cuenta la fuerza de la gravedad adicional a la resistencia al aire. 

$a_x = -\frac{D}{m}vv_x$

$a_y = -g-\frac{D}{m}vv_y$

**Posición considerando resistencia al aire**

Puesto a que la aceleración no es constante, es necesario encontrar los valores de posición, velocidad y aceleración en distintos instantes del tiempo. 

$\vec{s}(t + \Delta t) = \vec{s}(t) + \vec{v}(t)\Delta t + \frac{1}{2}\vec{a}(t)(\Delta t)^2$

**Expresión de Velocidad**

$\vec{v}(t + \Delta t) = v_x + \vec{a}_x\Delta t$

Las dos expresiones anteriores se deben separar en componentes.

**Expresión de masa**

$masa = densidad*volumen$

# Explicación de cálculos

In [70]:
import numpy as np
import matplotlib.pyplot as plt

**Función para encontrar máximos sin resistencia al aire**

In [71]:
def maximos(grados, v0):
    rad = np.radians(grados)
    v0y = v0*np.sin(rad)
    v0x = v0*np.cos(rad)
    c = [-4.9, v0y, y0]
    root = np.roots(c)
    for i in range(1):
        maxx= v0x*root[i]
    tmy = v0y/9.8
    maxy = -4.9*tmy**2 + v0y*tmy + y0
    return maxx, maxy

La función de máximos calcula primero las raíces de la función obtenida para el componente y de la trayectoria sin resistencia al aire. Una vez encontradas las raíces, encuentra la mayor que se refiere al tiempo total que tarda el proyectil para llegar al piso. Eso es multiplicado por la velocidad inicial en el componente x (dado que la velocidad en x se mantiene constante) y de esa manera se encuentra el alcance máximo. Asimismo, la altura máxima considera la derivada de la posición en el componente y; con el fin de calcular el tiempo que tarda en llegar al punto máximo, la derivada se iguala a 0 y se despeja t. Acto seguido, el tiempo es sustituido en la función del componente y para encontrar la altura máxima. 

**Cálculo de trayectoria sin resistencia al aire**

In [72]:
def no_drag(v0, grados, y0, t, g = 9.81):
    #Convertir ángulo a radianes
    rad = np.radians(grados)
    
    #Velocidad inicial
    v0x = v0 * np.cos(rad)
    v0y = v0 * np.sin(rad)

    #r(t)
    rx = v0x * t
    ry = y0 + v0y * t - 4.9 * t ** 2

    return rx , ry

La función del cálculo de la trayectoria sin resistencia al aire considera la velocidad inicial y sus componentes. Puesto a que en este caso no se considera que el componente x tiene un cambio en la aceleración, se considera que su velocidad (en el componente x) es constante. Por otro lado, se considera la fuerza de gravedad para el componente y por lo que la velocidad inicial en el componente y, varía.

**Datos considerando resistencia al aire**

In [73]:
def mass(r, densidad): 
    vol = (4/3) * np.pi * r ** 3
    m = vol * densidad
    return m

La función de masa calcula la masa del objeto a partir de su densidad y de su radio.

**Coeficiente D**

In [74]:
def coefficient(rho, c, a):
    d = (rho * c * a)/2
    return d

La función del coeficiente D utiliza la relación matemática para calcularlo a partir de la densidad del aire, el área frontal del objeto y el coeficiente de arrastre. 

**Función para encontrar puntos críticos**

In [91]:
def puntos_c(rx, ry):
    max_list = []
    m0 = (ry[1]-ry[0])/(rx[1]-rx[0])
    for i in range(2, len(ry)):
        m = (ry[i]-ry[i-1])/(rx[i]-rx[i-1])
        if (m0 > 0 and m < 0) or (m0 < 0 and m > 0):
            max_list.append(i)
        m0 = m
    return max_list

La función analiza las pendientes de la trayectoria sin resistencia al aire y determina la posición en la que se encuentra la altura máxima a partir de un cambio de signo en las pendientes (en este caso, de positiva a negativa).

**Función para encontrar el máximo en trayectoria con resistencia al aire**

In [92]:
def max_drag(x_list, y_list):
    ydmax = max(y_list)
    win = 0
    winner = 0
    for e in range (len(y_list)):
        ind = y_list[e]
        if ydmax == ind:
            win = e
        if ind < 0:
            winner = e-1
            break
    dragxm = x_list[win]
    dragym = y_list[win]
    xdmax = x_list[winner]
    return dragxm, dragym, xdmax

La función encuentra la posición en la que se encuentra la altura máxima y el alcance máximo. Asimismo, regresa sus coordenadas. Lo anterior lo hace buscando la posición del valor máximo en la lista de coordenadas x y y mediante diversos ciclos y condicionales. Después, busca el valor en las listas en dicha posición. 

**Cálculos trayectoria del proyectil con resistencia al aire**

In [77]:
def drag(v0, grados, y0, m, d, dt, itera):
    g = 9.81
    #Grados a radianes
    rad = np.radians(grados)

    # velocidad inicial
    v0x = v0 * np.cos(rad)
    v0y = v0 * np.sin(rad)

    #tiempo
    t = 0
    t_list = [t]

    #velocidad
    v = v0
    vx = v0x
    vy = v0y

    v_list = [v]
    v_x_list = [v0x]
    v_y_list = [v0y]

    # Posición
    x = 0
    y = y0

    x_list = [x]
    y_list = [y]

    # Aceleración inicial
    ax = -(d/m)*v*vx
    ay = -g-(d/m)*v*vy

    a_x_list = [ax]
    a_y_list = [ay]
    
    for _ in range(itera):
            
        # Velocidades para 'x' y 'y' del siguiente paso
        v_x_next = vx + (ax) * dt
        v_y_next = vy + (ay) * dt
        
        # Magnitud del vector de velocidad con componentes v_x_next y v_y_next
        v_next = np.sqrt((v_x_next) ** 2 + (v_y_next) ** 2)

        # Agregar valores a listas v_list, v_x_list, v_y_list. 
        v_list.append(v_next)
        v_x_list.append(v_x_next)
        v_y_list.append(v_y_next)

        # Posiciones 'x' y 'y' del siguiente paso
        x_next = x + v_x_next * dt + (1/2) * ax * (dt ** 2)
        y_next = y + v_y_next * dt + (1/2) * ay * (dt ** 2)
        
        # Agregar calores a listas x_list y y_list
        x_list.append(x_next)
        y_list.append(y_next)
        
        # Aceleraciones para 'x' y 'y' del siguiente paso
        a_x_next = -(d/m) * v * v_x_next
        a_y_next = -g -(d/m) * v * v_y_next

        # Agregar valores a listas a_x_list y a_y_list
        a_x_list.append(a_x_next)
        a_y_list.append(a_y_next)
        
        vx = v_x_next
        vy = v_y_next
        v = v_next

        x = x_next
        y = y_next

        ax = a_x_next
        ay = a_y_next

        # Calcular tiempo y guardarlo en una lista t_list
        t += dt
        t_list.append(t)
        
    return x_list, y_list, v_list, v_x_list, v_y_list, a_x_list, a_y_list, t_list

La función calcula tanto la posición como la velocidad y la aceleración en cada cambio del tiempo. Se consideran valores iniciales para calcular la velocidad inicial y sus componentes (iniciales), el tiempo, la posición inicial, y la aceleración inicial obtenida mediante la relación con la segunda ley de newton y la resistencia al aire incluida. El algoritmo para obtener las distintas cantidades físicas con el paso del tiempo consta de un ciclo que se repite una determinada cantidad de veces. Dicho ciclo calcula la velocidad en el siguiente instante del tiempo, la posición a partir de la "nueva" velocidad, y la aceleración en el siguiente instante del tiempo.

# Conclusión

Fue posible crear un programa que contribuya al análisis de balísticos y que sea flexible para cambiar diversas variables. Adicionalmente, fue posible modelar la trayectoria de un proyectil en dos situaciones, una sin incluir resistencia al aire, y la otra, incluyéndolo. 

Mediante la utilización de una aplicación que incluye tanto las consideraciones físicas del movimiento como los algoritmos mencionados en el presente escrito, es posible brindar una simulación útil para considerar variaciones en el comportamiento de las erupciones volcánicas. Esto, tiene una posible contribución al entrenamiento de rescatistas y a la evaluación de decisiones en una situación de actividad volcánica. 

En cuanto a las limitaciones del presente escrito, una de gran importancia es el tomar al proyectil como una esfera perfecta, que no es el caso del fenómeno.

# Referencias

Alatorre, M. (2011). A model of volcanic explosions at Popocatépetl volcano (Mexico)... [Sitio Web]. Recuperado de: https://d-nb.info/1015203132/34

Becerra, L., Guardado, M. (2001). Estimación de la incertidumbre en la determinación de la densidad del aire. CENAM. [Sitio Web]. Recuperado de: http://www.cenam.mx/myd/DENSIDAD%20DEL%20AIRE%20abril-20031.pdf

Bucheli, E. Modelación Computacional del Movimiento. Tecnológico de Monterrey, Ciudad de México. 7 oct. 2020

Katsuda, A. (2020). Entregable 2. Tecnológico de Monterrey, Ciudad de México. 

INEGI (2017). Anuario estadístico y geográfico de Puebla 2017. [Sitio Web]. Recuperado de: 
https://www.datatur.sectur.gob.mx/ITxEF_Docs/PUE_ANUARIO_PDF.pdf

Pacheco, J. (2008) Análisis y Comparación de las emisiones… UNAM. [Sitio Web]. Recuperado de: http://www.ptolomeo.unam.mx:8080/xmlui/bitstream/handle/132.248.52.100/8070/Tesis_Completa.pdf?sequence=1	

Shiffman, D. (s.f.). Resistencia del aire y de fluidos. Khan Academy. [Sitio Web]. Recuperado de: https://es.khanacademy.org/computing/computer-programming/programming-natural-simulations/programming-forces/a/air-and-fluid-resistance		

Yunus, Ç., Cimbala, J. (2006). Mecánica de fluidos: Fundamentos y aplicaciones. [Sitio Web]. Recuperado de: http://tesis.pucp.edu.pe/repositorio/bitstream/handle/20.500.12404/5421/SOTOMAYOR_DENIS_SIMULACION_NUMERICA_INTERCAMBIADOR_CALOR_FLUJO_TRANSVERSAL_ALETEADO_ANEXOS.pdf?sequence=2&isAllowed=y